In [28]:
# ==============================================================================
# CELL 1: CÀI ĐẶT, IMPORTS & THIẾT LẬP BAN ĐẦU (v3.0)
# ==============================================================================
import os
import json
import asyncio
from typing import TypedDict, Annotated, List, Dict, Any, Literal, Optional
import operator

# Tải các biến môi trường
from dotenv import load_dotenv 
load_dotenv()

# LangChain & LangGraph
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# Thư viện hỗ trợ & Kho tri thức
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from rich.console import Console
from rich.markdown import Markdown
from rich.pretty import pprint
from IPython.display import Image, display

# --- CẤU HÌNH LOG ---
VERBOSE_MODE = True

# --- KHỞI TẠO CÁC CÔNG CỤ ---
console = Console()

def print_step(message: str):
    if VERBOSE_MODE:
        console.print(f"[bold cyan]>[/bold cyan] {message}")

def print_result(data: Any, title: str = "Kết quả"):
    if VERBOSE_MODE:
        console.print(f"[bold green]✔️ {title}:[/bold green]")
        pprint(data, expand_all=True)
        
def print_warning(message: str):
    if VERBOSE_MODE:
        console.print(f"[bold yellow]⚠️  {message}[/bold yellow]")

# --- KHỞI TẠO LLM ---
try:
    llm = ChatGroq(
        temperature=0.1, model="llama3-70b-8192",
        api_key=os.getenv("GROQ_API_KEY"), max_tokens=3072
    )
    print("✅ LLM (Groq) đã được khởi tạo thành công.")
except Exception as e:
    print(f"❌ LỖI: Không thể khởi tạo LLM. Lỗi: {e}")
    llm = None

# --- TẢI KHO TRI THỨC ---
VECTOR_STORE_PATH = "vector_store/sgk_toan_9"
if os.path.exists(VECTOR_STORE_PATH):
    try:
        embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        vector_store = FAISS.load_local(VECTOR_STORE_PATH, embeddings, allow_dangerous_deserialization=True)
        print("✅ Kho tri thức RAG đã được tải thành công.")
    except Exception as e:
        print(f"❌ LỖI khi tải kho tri thức: {e}")
        vector_store = None
else:
    print(f"❌ LỖI: Không tìm thấy kho tri thức tại '{VECTOR_STORE_PATH}'.")
    vector_store = None

✅ LLM (Groq) đã được khởi tạo thành công.
✅ Kho tri thức RAG đã được tải thành công.


In [29]:
# ==============================================================================
# CELL 2: NÂNG CẤP STATE ĐỂ CÓ "NHẬN THỨC TOÀN CỤC" (STATE V4)
# ==============================================================================
from typing import TypedDict, Annotated, List, Dict, Any, Literal, Optional
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
import operator

# --- Hàm tiện ích (Giữ nguyên) ---
def merge_dicts(dict1: Dict[str, Any], dict2: Dict[str, Any]) -> Dict[str, Any]:
    # ... (code giữ nguyên)
    merged = dict1.copy()
    for key, value in dict2.items():
        if key in merged and isinstance(merged[key], dict) and isinstance(value, dict):
            merged[key] = merge_dicts(merged[key], value)
        else:
            merged[key] = value
    return merged

# --- Cấu trúc dữ liệu con (Giữ nguyên) ---
class DetailedTask(TypedDict):
    task_id: int
    task_name: str
    task_description: str
    estimated_duration: int 
    status: Literal["pending", "completed"]

# --- State chính phiên bản 4.0 (cho workflow v6.0) ---
class TeacherStateV4(TypedDict):
    # Lấy ra yêu cầu gốc từ tin nhắn và lưu lại
    original_request: Annotated[List[BaseMessage], operator.itemgetter("messages"), lambda x: x[-1].content]
    messages: Annotated[List[BaseMessage], add_messages]
    
    # Các bước tuần tự ban đầu
    analyzed_objective: Optional[Any]
    pedagogy_strategy: Optional[Any]
    
    # === NÂNG CẤP LỚN ===
    # Thêm trường để lưu các truy vấn RAG đã được mở rộng
    expanded_queries: Optional[List[str]]
    
    # Danh sách nhiệm vụ, Critic notes, outputs (giữ nguyên)
    task_list: Optional[List[DetailedTask]]
    current_task_id: Optional[int]
    reflection_notes: Optional[str]
    agent_outputs: Annotated[Dict[str, Any], merge_dicts] 
    final_lesson_plan: Optional[str]
    next_agent: str

print("✅ 'TeacherStateV4' với Nhận thức Toàn cục đã được định nghĩa.")

✅ 'TeacherStateV4' với Nhận thức Toàn cục đã được định nghĩa.


In [30]:
# ==============================================================================
# CELL 3 (Hoàn chỉnh v6.0): "ĐẠI PHẪU" CÁC AGENT
# ==============================================================================
import json
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict, Any, Literal, Optional

# --- ĐỊNH NGHĨA CÁC CẤU TRÚC PARSE Pydantic ---
# (Giữ nguyên các model đã ổn định, thêm model cho các agent mới)
class ExpandedQueries(BaseModel):
    queries: List[str] = Field(description="Danh sách các truy vấn tìm kiếm cụ thể và đa dạng.")
class BestSnippets(BaseModel):
    best_snippets: List[str] = Field(description="Danh sách các đoạn văn bản được chọn lọc, phù hợp nhất với yêu cầu.")
# ... (Các model khác như ParsedObjective, LearningActivity, etc. giữ nguyên)
class ParsedObjective(BaseModel):
    action_verb: str; bloom_level: int; topic: str; grade_level: str; duration_minutes: Optional[int]
class PedagogyChoice(BaseModel):
    chosen_pedagogy: str; pedagogy_rationale: str
class DetailedTaskModel(BaseModel):
    task_name: str; task_description: str; estimated_duration: int
class TaskListWithDuration(BaseModel):
    tasks: List[DetailedTaskModel]
class TaskClassification(BaseModel):
    agent_category: Literal["activity_designer", "theory_synthesizer", "assessment_creator"]
class LearningActivity(BaseModel):
    activity_name: str; description: str; duration_minutes: int; activity_type: str
class AssessmentItem(BaseModel):
    question: str; question_type: str; options: Optional[List[str]]; answer: str


# --- HÀM TIỆN ÍCH ---
def update_agent_outputs(state: "TeacherStateV4", key: str, value: Any) -> Dict[str, Any]:
    outputs = state.get("agent_outputs", {}).copy()
    if key not in outputs: outputs[key] = []
    outputs[key].append(value)
    return {"agent_outputs": outputs}

# === CHỈ THỊ CẤP HỆ THỐNG (Direction 3 - Hoàn thiện Persona) ===
SYSTEM_PERSONA_PROMPT = """
BẠN LÀ MỘT TRỢ LÝ AI CHUYÊN NGHIỆP, ĐÓNG VAI TRÒ MỘT GIÁO VIÊN TOÁN LỚP 9 GIÀU KINH NGHIỆM TẠI VIỆT NAM.
Hãy tưởng tượng bạn đang soạn giáo án này vào tối Chủ Nhật để dạy vào sáng thứ Hai. Nó cần phải chi tiết, thực tế và sẵn sàng để in ra mang vào lớp.
QUY TẮC BẮT BUỘC:
1.  **NGÔN NGỮ:** Bạn PHẢI trả lời HOÀN TOÀN bằng TIẾNG VIỆT.
2.  **CHUYÊN MÔN:** Sử dụng thuật ngữ toán học và sư phạm chính xác, phù hợp với chương trình giáo dục Việt Nam.
3.  **THỰC TẾ:** Mọi ví dụ và hoạt động bạn tạo ra phải thực tế, gần gũi và phù hợp với học sinh lớp 9.
"""

# --- CÁC AGENTS PHIÊN BẢN 6.0 ---

# ObjectiveInterpreter và PedagogyStrategist được tiêm Persona mới
async def objective_interpreter_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    #... (Logic tương tự, chỉ thêm SYSTEM_PERSONA_PROMPT)
    print_step("`Agent: Objective Interpreter` (v6.0) đang phân tích mục tiêu...")
    prompt = f"{SYSTEM_PERSONA_PROMPT}\n**YÊU CẦU NGƯỜI DÙNG:** \"{state['original_request']}\"\n**NHIỆM VỤ:** Phân tích và trích xuất thông tin vào JSON..."
    structured_llm = llm.with_structured_output(ParsedObjective, method="json_mode")
    # ... try/except
    parsed_result = await structured_llm.ainvoke(prompt)
    analyzed_objective_dict = parsed_result.dict()
    analyzed_objective_dict['constraints'] = {'duration_minutes': parsed_result.duration_minutes}
    del analyzed_objective_dict['duration_minutes']
    print_result(analyzed_objective_dict, "Mục tiêu & Ràng buộc đã phân tích")
    return {"analyzed_objective": analyzed_objective_dict}


async def pedagogy_strategist_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    #... (Logic tương tự, chỉ thêm SYSTEM_PERSONA_PROMPT)
    print_step("\n---\n### 🧑‍🏫 `Agent: Pedagogy Strategist` (v6.0) Bắt đầu...")
    prompt = f"{SYSTEM_PERSONA_PROMPT}\n**MỤC TIÊU:** {state['analyzed_objective']}\n**NHIỆM VỤ:** Chọn phương pháp sư phạm và giải thích. Trả về JSON."
    #...
    structured_llm = llm.with_structured_output(schema=PedagogyChoice, method="json_mode")
    response = await structured_llm.ainvoke(prompt)
    return {"pedagogy_strategy": response.dict()}


# === AGENT MỚI (Direction 1 - Query Transformation) ===
async def query_expansion_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    """Phân rã yêu cầu chính thành các truy vấn con cụ thể để tìm kiếm RAG hiệu quả hơn."""
    print_step("\n---\n### 🧠 `Agent: Query Expansion` (v6.0) đang phân tích yêu cầu để tìm kiếm tốt hơn...")
    objective = state['analyzed_objective']
    prompt = f"""{SYSTEM_PERSONA_PROMPT}
    Bạn là một trợ lý nghiên cứu thông minh. Dựa vào mục tiêu bài học sau, hãy tạo ra một danh sách các từ khóa và cụm từ tìm kiếm đa dạng để lấy được thông tin chính xác nhất từ sách giáo khoa.
    **MỤC TIÊU BÀI HỌC:** "{objective['topic']}" và kỹ năng "{objective['action_verb']}".
    **VÍ DỤ:** Nếu mục tiêu là "Vận dụng góc nội tiếp và cung bị chắn", các truy vấn có thể là: "định nghĩa góc nội tiếp", "định lý số đo góc nội tiếp", "hệ quả góc nội tiếp", "chứng minh tứ giác nội tiếp bằng cung chứa góc".
    **YÊU CẦU JSON:** Trả về một đối tượng JSON với key duy nhất là `queries`, giá trị là một danh sách các chuỗi truy vấn.
    """
    structured_llm = llm.with_structured_output(ExpandedQueries, method="json_mode")
    response = await structured_llm.ainvoke(prompt)
    print_result(response.queries, "Các truy vấn tìm kiếm đã được mở rộng")
    return {"expanded_queries": response.queries}

# === AGENT ĐƯỢC NÂNG CẤP LỚN (Direction 1 - Re-ranking) ===
async def resource_scout_agent_v2(state: "TeacherStateV4") -> Dict[str, Any]:
    """Sử dụng các truy vấn đã mở rộng để tìm kiếm, sau đó tái xếp hạng kết quả để chọn ra thông tin phù hợp nhất."""
    print_step("\n---\n### 📚 `Agent: Resource Scout` (v6.0) đang thực hiện tìm kiếm và sàng lọc thông tin...")
    retriever = vector_store.as_retriever(search_kwargs={"k": 2})
    
    # 1. Lấy tài liệu từ tất cả các truy vấn con
    all_docs = []
    for query in state['expanded_queries']:
        all_docs.extend(retriever.invoke(query))
    
    # Loại bỏ các tài liệu trùng lặp
    unique_docs_content = list({doc.page_content for doc in all_docs})
    print_step(f"Tìm thấy tổng cộng {len(unique_docs_content)} đoạn văn bản độc nhất.")

    # 2. Tái xếp hạng (Re-ranking)
    rerank_prompt = f"""{SYSTEM_PERSONA_PROMPT}
    **YÊU CẦU GỐC CỦA GIÁO VIÊN:** "{state['original_request']}"
    **DANH SÁCH CÁC ĐOẠN VĂN BẢN TÌM ĐƯỢC TỪ SÁCH GIÁO KHOA:**
    ---
    {json.dumps(unique_docs_content, ensure_ascii=False, indent=2)}
    ---
    **NHIỆM VỤ:** Đọc kỹ YÊU CẦU GỐC và chọn ra 3-4 đoạn văn bản phù hợp NHẤT để xây dựng giáo án.
    **YÊU CẦU JSON:** Trả về một đối tượng JSON với key `best_snippets` chứa danh sách các đoạn văn bản đã chọn.
    """
    structured_llm_reranker = llm.with_structured_output(BestSnippets, method="json_mode")
    reranked_result = await structured_llm_reranker.ainvoke(rerank_prompt)
    best_snippets_text = "\n\n---\n\n".join(reranked_result.best_snippets)
    print_step("Đã sàng lọc và chọn ra các đoạn văn bản phù hợp nhất.")

    # 3. Tóm tắt thông tin cuối cùng
    summary_prompt = f"Tóm tắt các kiến thức, định nghĩa, công thức cốt lõi từ các đoạn văn bản sau BẰNG TIẾNG VIỆT:\n\n{best_snippets_text}"
    summary_response = await llm.ainvoke(summary_prompt)
    
    resource = {"source": "Sách giáo khoa Toán 9 (từ RAG - đã sàng lọc)", "summary": summary_response.content}
    print_result(resource, "Tài liệu RAG cuối cùng")
    return update_agent_outputs(state, "resources", resource)

# === CÁC AGENT SÁNG TẠO ĐƯỢC NÂNG CẤP (Direction 2 - Nhận thức toàn cục) ===
async def plan_delegator_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("\n---\n### 👷 `Agent: Plan Delegator` (v6.0) đang lập kế hoạch chiến lược...")
    prompt = f"""{SYSTEM_PERSONA_PROMPT}
    **BỐI CẢNH TOÀN CỤC:**
    - Yêu cầu gốc của giáo viên: "{state['original_request']}"
    - Mục tiêu đã phân tích: {state['analyzed_objective']}
    - Kiến thức nền có sẵn: {state['agent_outputs']['resources'][0]['summary']}
    {f"- PHẢN HỒI TỪ LẦN TRƯỚC: {state['reflection_notes']}" if state.get('reflection_notes') else ""}

    **NHIỆM VỤ:** Lập kế hoạch chi tiết cho buổi học.
    **QUY TẮC:**
    1.  **KIỂM TRA CHÉO:** Kế hoạch bạn tạo ra phải TRỰC TIẾP phục vụ Yêu cầu gốc và Mục tiêu đã phân tích.
    2.  Luôn bắt đầu bằng hoạt động ôn tập lý thuyết.
    3.  Các hoạt động sau phải tập trung vào vận dụng kiến thức có sẵn.
    4.  Luôn kết thúc bằng hoạt động đánh giá.
    **YÊU CẦU JSON:** ... (như cũ)
    """
    # ... (logic thực thi và try/except như cũ)
    structured_llm = llm.with_structured_output(TaskListWithDuration, method="json_mode")
    task_list_result = await structured_llm.ainvoke(prompt)
    tasks_with_status = [{"task_id": i, **t.dict(), "status": "pending"} for i, t in enumerate(task_list_result.tasks) if t]
    print_result(tasks_with_status, f"Đã tạo {len(tasks_with_status)} nhiệm vụ chiến lược")
    return {"task_list": tasks_with_status, "reflection_notes": None, "agent_outputs": {}}

# ... (Các agent khác như TheorySynthesizer, ActivityDesigner, AssessmentCreator, Compiler cũng được thêm vào
#      phần "BỐI CẢNH TOÀN CỤC" và "QUY TẮC KIỂM TRA CHÉO" tương tự)
# Ví dụ cho ActivityDesigner:
async def activity_designer_agent(state: "TeacherStateV4") -> Dict[str, Any]:
    print_step("`Agent: Activity Designer` (v6.0) đang thiết kế hoạt động VẬN DỤNG CỤ THỂ...")
    task_to_run = next((t for t in state['task_list'] if t['task_id'] == state['current_task_id']), None)
    prompt = f"""{SYSTEM_PERSONA_PROMPT}
    **BỐI CẢNH TOÀN CỤC:**
    - Yêu cầu gốc của giáo viên: "{state['original_request']}"
    - Mục tiêu bài học: {state['analyzed_objective']}

    **NHIỆM VỤ CỤ THỂ:** "{task_to_run['task_description']}"
    **THỜI GIAN:** {task_to_run['estimated_duration']} phút.

    **QUY TẮC SÁNG TẠO:**
    1.  **KIỂM TRA CHÉO:** Hoạt động bạn tạo ra phải TRỰC TIẾP phục vụ Yêu cầu gốc và Mục tiêu bài học.
    2.  **TÍNH CỤ THỂ:** Phải có bối cảnh và số liệu cụ thể, rõ ràng.
    3.  **TƯ DUY PHẢN BIỆN:** Tự kiểm tra xem bài toán có logic, thực tế và giải được không. Nếu không, phải làm lại.
    
    **YÊU CẦU JSON:** Trả về JSON PHẲNG với keys: `activity_name`, `description`, `duration_minutes`, `activity_type`.
    """
    # ... (logic thực thi và try/except như cũ)
    structured_llm = llm.with_structured_output(LearningActivity, method="json_mode")
    activity_result = await structured_llm.ainvoke(prompt)
    print_result(activity_result.dict(), "Hoạt động vận dụng đã thiết kế")
    return update_agent_outputs(state, "activities", activity_result.dict())

print("✅ Tất cả các agent đã được nâng cấp lên phiên bản 6.0.")

✅ Tất cả các agent đã được nâng cấp lên phiên bản 6.0.


In [31]:
# ==============================================================================
# CELL 4 (Hoàn chỉnh v6.0): LẮP RÁP GRAPH VỚI LUỒNG RAG NÂNG CAO
# ==============================================================================
from typing import Literal, Dict, Any

# --- Node tiện ích (Không đổi) ---
def mark_task_complete(state: "TeacherStateV4") -> Dict[str, Any]: #...
def task_router_node(state: "TeacherStateV4") -> Dict[str, Any]: #...
async def task_dispatcher_agent(state: "TeacherStateV4") -> Dict[str, Any]: #...
def route_after_router(state: "TeacherStateV4") -> Literal["continue_executing", "compile_and_critique"]: #...
def route_after_compilation(state: "TeacherStateV4") -> Literal["finish", "replan"]: #...

# --- XÂY DỰNG GRAPH PHIÊN BẢN 6.0 ---
workflow = StateGraph(TeacherStateV4)

# 1. Thêm tất cả các node vào graph, bao gồm cả các agent mới/nâng cấp
workflow.add_node("objective_interpreter", objective_interpreter_agent)
workflow.add_node("pedagogy_strategist", pedagogy_strategist_agent)
workflow.add_node("query_expansion", query_expansion_agent) # Agent mới
workflow.add_node("resource_scout", resource_scout_agent_v2) # Agent nâng cấp
workflow.add_node("plan_delegator", plan_delegator_agent)
workflow.add_node("task_router", task_router_node)
workflow.add_node("task_dispatcher", task_dispatcher_agent)
workflow.add_node("theory_synthesizer", theory_synthesizer_agent)
workflow.add_node("activity_designer", activity_designer_agent)
workflow.add_node("assessment_creator", assessment_creator_agent)
workflow.add_node("mark_task_complete", mark_task_complete)
workflow.add_node("plan_compiler_and_critic", plan_compiler_and_critic_agent)

# 2. Kết nối các bước tuần tự ban đầu theo luồng RAG mới
workflow.set_entry_point("objective_interpreter")
workflow.add_edge("objective_interpreter", "pedagogy_strategist")
workflow.add_edge("pedagogy_strategist", "query_expansion") # -> Mở rộng truy vấn
workflow.add_edge("query_expansion", "resource_scout") # -> Tìm kiếm và sàng lọc
workflow.add_edge("resource_scout", "plan_delegator") # -> Lập kế hoạch với kiến thức ĐÚNG

# 3. Phần còn lại của graph (vòng lặp task, vòng lặp critic) giữ nguyên
workflow.add_edge("plan_delegator", "task_router")
workflow.add_conditional_edges(
    "task_router",
    route_after_router,
    {"continue_executing": "task_dispatcher", "compile_and_critique": "plan_compiler_and_critic"}
)
workflow.add_conditional_edges(
    "task_dispatcher",
    lambda state: state["next_agent"],
    {"activity_designer": "activity_designer", "assessment_creator": "assessment_creator", "theory_synthesizer": "theory_synthesizer"}
)
workflow.add_edge("activity_designer", "mark_task_complete")
workflow.add_edge("assessment_creator", "mark_task_complete")
workflow.add_edge("theory_synthesizer", "mark_task_complete")
workflow.add_edge("mark_task_complete", "task_router")
workflow.add_conditional_edges(
    "plan_compiler_and_critic",
    route_after_compilation,
    {"replan": "plan_delegator", "finish": END}
)

# 4. Biên dịch Graph
app = workflow.compile()
print("✅ Graph 6.0 (với Trí tuệ Sư phạm Nâng cao) đã được biên dịch thành công.")

# 5. Vẽ biểu đồ
try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Lỗi vẽ biểu đồ: {e}")

IndentationError: expected an indented block after function definition on line 7 (2978117507.py, line 8)

In [ ]:
# ==============================================================================
# CELL 5: CHẠY THỬ NGHIỆM VỚI GRAPH V3.0
# ==============================================================================

async def run_graph_v3(user_request: str):
    initial_state = {
        "messages": [HumanMessage(content=user_request)],
        "agent_outputs": {},
    }
    console.print(f"\n[bold magenta]🚀 BẮT ĐẦU QUY TRÌNH V3.0 VỚI YÊU CẦU:[/bold magenta]\n> {user_request}")
    config = {"recursion_limit": 100} # Tăng giới hạn đệ quy để cho phép vòng lặp
    
    final_state_result = None
    async for event in app.astream(initial_state, config=config):
        for node_name, node_output in event.items():
            console.print(f"\n[bold yellow]------- Hoàn thành bước: {node_name} -------[/bold yellow]")
            final_state_result = node_output

    console.print("\n[bold magenta]🏁 KẾT THÚC QUY TRÌNH V3.0.[/bold magenta]")
    if final_state_result and final_state_result.get("final_lesson_plan"):
        console.print("\n[bold green]📝 GIÁO ÁN HOÀN CHỈNH (SAU KHI ĐÃ TỰ SỬA LỖI)[/bold green]")
        console.print(Markdown(final_state_result["final_lesson_plan"]))
    else:
        print_warning("Không tạo được giáo án cuối cùng. Có thể quy trình đã dừng lại do lỗi không thể khắc phục.")

# Yêu cầu cũ, nhưng kỳ vọng vào kết quả mới, tốt hơn
request_v3_test = "Soạn giúp tôi giáo án bài 'Đường tròn ngoại tiếp và đường tròn nội tiếp' cho học sinh lớp 9. Mục tiêu là các em có thể vận dụng được góc nội tiếp và cung bị chắn để giải các bài toán trong phần luyện tập. Buổi học sẽ diễn ra trong 2 tiết, tổng cộng 90 phút."

# Bật log chi tiết để theo dõi quá trình
VERBOSE_MODE = True 

# Chạy thử
await run_graph_v3(request_v3_test)

🚀 BẮT ĐẦU QUY TRÌNH V3.0 VỚI YÊU CẦU:
> Soạn giúp tôi giáo án bài 'Đường tròn ngoại tiếp và đường tròn nội tiếp' cho học sinh lớp 9. Mục tiêu là các em 
có thể vận dụng được góc nội tiếp và cung bị chắn để giải các bài toán trong phần luyện tập. Buổi học sẽ diễn ra 
trong 2 tiết, tổng cộng 90 phút.

> `Agent: Objective Interpreter` (v5.0) đang phân tích mục tiêu...

✔️ Mục tiêu & Ràng buộc đã phân tích:

{
│   'action_verb': 'vận dụng',
│   'bloom_level': 3,
│   'topic': 'Đường tròn ngoại tiếp và đường tròn nội tiếp',
│   'grade_level': '9',
│   'constraints': {
│   │   'duration_minutes': 90
│   }
}

------- Hoàn thành bước: objective_interpreter -------

...

> ROUTER: Critic đã hài lòng. Kết thúc quy trình.

------- Hoàn thành bước: plan_compiler_and_critic -------

🏁 KẾT THÚC QUY TRÌNH V3.0.

📝 GIÁO ÁN HOÀN CHỈNH (SAU KHI ĐÃ TỰ SỬA LỖI)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                              GIÁO ÁN: Đường tròn ngoại tiếp và đường tròn nội tiếp                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛


                                                I. THÔNG TIN CHUNG                                                 

 • Môn học: Toán                                                                                                   
 • Lớp: 9                                                                                                          
 • Thời lượng dự kiến: 90 phút                                                                                     


                                               II. MỤC TIÊU BÀI HỌC                                                

 • Học sinh có thể vận dụng được các tỉ số lượng giác để giải các bài toán thực tế.                                
 • Rèn luyện kỹ năng mô hình hóa toán học và giải quyết vấn đề.                                                    


                                              III. TIẾN TRÌNH DẠY HỌC                                              

                    Hoạt động 1: Ôn tập đường tròn ngoại tiếp và đường tròn nội tiếp (15 phút)                     

 • Loại hình tổ chức: Trắc nghiệm nhanh                                                                            
 • Mô tả chi tiết: Hoàn thành các câu hỏi trắc nghiệm nhanh để ôn tập lại các công thức và định nghĩa về đường tròn
   ngoại tiếp và đường tròn nội tiếp.                                                                              

                Hoạt động 2: Tính chu vi và diện tích của đường tròn trong thiết kế logo (20 phút)                 

 • Loại hình tổ chức: Bài toán tính chu vi và diện tích của đường tròn                                             
 • Mô tả chi tiết: Công ty thiết kế logo cần tạo một logo hình tròn cho một công ty mới. Họ muốn logo có bán kính  
   là 4cm. Hãy tính chu vi và diện tích của logo này.                                                              

      Hoạt động 3: Tìm tâm và bán kính của đường tròn ngoại tiếp tam giác trong thiết kế kiến trúc (25 phút)       

 • Loại hình tổ chức: Bài toán thực tế                                                                             
 • Mô tả chi tiết: Một kiến trúc sư đang thiết kế một công trình xây dựng và cần tìm tâm và bán kính của đường tròn
   ngoại tiếp một tam giác có các cạnh là 6m, 8m và 10m. Hãy giúp kiến trúc sư tìm tâm và bán kính của đường tròn  
   đó.                                                                                                             

                   Hoạt động 4: Tính toán đường tròn ngoại tiếp và đường tròn nội tiếp (30 phút)                   

 • Loại hình tổ chức: Bài toán tổng hợp về đường tròn ngoại tiếp và đường tròn nội tiếp                            
 • Mô tả chi tiết: Một công ty sản xuất bánh xe đạp cần tính toán đường tròn ngoại tiếp và đường tròn nội tiếp của 
   bánh xe để sản xuất bánh xe đạt tiêu chuẩn. Cho hình tròn có chu vi là 16π cm. Xác định bán kính và diện tích   
   của hình tròn.                                                                                                  


                                              IV. ĐÁNH GIÁ VÀ CỦNG CỐ                                              


                                            V. TÀI NGUYÊN & PHƯƠNG TIỆN                                            

 • Nguồn: Sách giáo khoa Toán 9 (từ RAG)                                                                           
 • Tóm tắt kiến thức cốt lõi: Here is a summary of the key concepts, definitions, and formulas related to "Tangent 
   and Secant Lines" from the provided text:                                                                       

Definitions:                      